<a href="https://colab.research.google.com/github/BradenAnderson/sales-predictions/blob/main/05_Interpretable_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Braden Anderson
## Sales Predictions Project part 5
## Interpretable (Linear) Modeling

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import missingno
import pickle
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, BaggingRegressor 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score, mean_squared_error, SCORERS
from sklearn.pipeline import Pipeline


%matplotlib inline

In [ ]:
filename = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/Sales_Training_Data_Cleaned_Imputed.csv'

sales_df = pd.read_csv(filename)

In [ ]:
sales_df.drop(columns=['Item_Identifier', 'Outlet_Establishment_Year'], inplace=True)

In [ ]:

nominal_categorical_feat = ['Item_Type', 'Outlet_Identifier', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Id_Binned']
ordinal_categorical_feat = ['Item_Fat_Content', 'Outlet_Size', 'Years_of_operation']
numeric_feat = ['Item_Weight', 'Item_Visibility', 'Item_MRP']

X = sales_df.loc[:, sales_df.columns != 'Item_Outlet_Sales']
y = sales_df.loc[:, 'Item_Outlet_Sales']


preprocess = ColumnTransformer(transformers=[("one_hot_encoder", OneHotEncoder(drop="first", sparse=False), nominal_categorical_feat),
                                             ("ordinal_encoder", OrdinalEncoder(categories=[['Not_Edible', 'Low Fat', 'Regular'], ['Small', 'Medium', 'High'],
                                                                                            [4, 6, 9, 11, 14, 15, 16, 26, 28]]), ordinal_categorical_feat)],
                               remainder='passthrough')



column_list = list(range(5,38))

reg_strategy = TransformedTargetRegressor()
bcox_transformer = PowerTransformer(method='box-cox')

model_pipeline = Pipeline([("preprocessing", preprocess),
                           ('feature_selector', SelectKBest(f_regression)),
                           ('regress', reg_strategy)])

parameter_grid = [{'feature_selector__k' : column_list,
                   'regress__regressor' : [LinearRegression(), Lasso(), Ridge()],
                   'regress__regressor__fit_intercept' : [False, True],
                   'regress__transformer' : [None, bcox_transformer]}]


score_types = {'r2' :'r2', 'MSE' : 'neg_mean_squared_error'}

gs = GridSearchCV(estimator=model_pipeline, param_grid=parameter_grid, scoring=score_types, refit='MSE', cv=5, n_jobs=-1)

gs.fit(X, y)

PATH = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/'
gridsearch_result_filename = 'gridsearch_models_5_Linear.pkl'
full_path = PATH + gridsearch_result_filename
with open(full_path, 'wb') as file:
  pickle.dump(gs, file)


In [ ]:
full_path = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/gridsearch_models_5_Linear.pkl'

with open(full_path, 'rb') as file:
  gs_results = pickle.load(file)

search_results = gs_results.cv_results_
top_estimator = gs_results.best_estimator_
top_score = gs_results.best_score_
top_parameters = gs_results.best_params_

gs_result_df = pd.DataFrame(search_results)

In [51]:
results_simplified_df= gs_result_df.loc[:,:].sort_values(by=['rank_test_MSE'], ignore_index=True)

results_simplified_df.rename(columns= {'param_feature_selector__k' : 'num_features_in_model',
                                       'param_regress__regressor' : 'model_type',
                                       'param_regress__transformer' : 'target_transformation',
                                       'param_regress__regressor__fit_intercept' : 'fit_intercept'}, inplace=True)

results_simplified_df = results_simplified_df.loc[:, ['num_features_in_model', 'model_type', 'target_transformation', 'fit_intercept',
                                                      'params', 'mean_test_r2', 'rank_test_r2', 'mean_test_MSE', 'rank_test_MSE']]


results_simplified_df['target_transformation'] = results_simplified_df['target_transformation'].fillna(value="No_Transformation")
results_simplified_df.loc[ (results_simplified_df['target_transformation'] != 'No_Transformation') , 'target_transformation'] = "Box-Cox"

# For easier viewing, trim model_type down so it only shows the name of the model used.
results_simplified_df['model_type'] = results_simplified_df['model_type'].astype(str)
results_simplified_df['model_type'] = results_simplified_df['model_type'].map(lambda model_string : model_string.split('(')[0])

# Negative MSE is not very interpretable. Take the absolute value and square root to get a more meaningful number. 
results_simplified_df['RMSE'] = results_simplified_df['mean_test_MSE'].abs().pow(1./2)

pd.set_option('display.max_rows', 500)

results_simplified_df.head(50)


,num_features_in_model,model_type,target_transformation,fit_intercept,params,mean_test_r2,rank_test_r2,mean_test_MSE,rank_test_MSE,RMSE
0,16,LinearRegression,Box-Cox,True,"{'feature_selector__k': 16, 'regress__regresso...",0.581160,1,-1.218403e+06,1,1103.812926
1,17,LinearRegression,Box-Cox,True,"{'feature_selector__k': 17, 'regress__regresso...",0.581146,2,-1.218451e+06,2,1103.834566
2,17,LinearRegression,Box-Cox,False,"{'feature_selector__k': 17, 'regress__regresso...",0.581146,3,-1.218451e+06,3,1103.834566
3,18,LinearRegression,Box-Cox,True,"{'feature_selector__k': 18, 'regress__regresso...",0.581100,4,-1.218591e+06,4,1103.898306
4,18,LinearRegression,Box-Cox,False,"{'feature_selector__k': 18, 'regress__regresso...",0.581100,5,-1.218591e+06,5,1103.898306
5,16,Ridge,Box-Cox,True,"{'feature_selector__k': 16, 'regress__regresso...",0.581058,6,-1.218700e+06,6,1103.947666
6,17,Ridge,Box-Cox,True,"{'feature_selector__k': 17, 'regress__regresso...",0.581042,7,-1.218753e+06,7,1103.971297
7,18,Ridge,Box-Cox,True,"{'feature_selector__k': 18, 'regress__regresso...",0.580997,8,-1.218893e+06,8,1104.035055
8,15,LinearRegression,Box-Cox,True,"{'feature_selector__k': 15, 'regress__regresso...",0.580945,9,-1.219014e+06,9,1104.089725
9,15,Ridge,Box-Cox,True,"{'feature_selector__k': 15, 'regress__regresso...",0.580849,10,-1.219294e+06,10,1104.216623


In [ ]:
# Thank you Joey Gao!! 
# https://github.com/scikit-learn/scikit-learn/issues/12525

def get_column_names_from_ColumnTransformer(column_transformer):    

    col_name = []

    for transformer_in_columns in column_transformer.transformers_:#the last transformer is ColumnTransformer's 'remainder' # <--- I modified this. In my case the last transformer was not the remainder. It was the ordinal encoder.

        raw_col_name = transformer_in_columns[2]

        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name

        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)
            
    return col_name

In [ ]:
# Get a boolean array that indicates which features SelectKBest chose when the best model was being fit.
best_model_features_selected = gs_results.best_estimator_.named_steps['feature_selector'].get_support()

# Get the column transformer instance used when creating the best model. 
best_model_ColumnTransformer =  gs_results.best_estimator_.named_steps['preprocessing']

# Get the best models column names from the column transformer. 
best_model_column_names = get_column_names_from_ColumnTransformer(best_model_ColumnTransformer)

# Convert the best models column names to a np.array so we can index the list using the boolean matrix
# that indicates which features were used when fitting the best model. 
best_model_column_names = np.array(best_model_column_names)

# Get the names of the features used in the best model.
best_model_feature_names = best_model_column_names[best_model_features_selected]

# Convert the names of best features from a numpy array back to a python list.
best_model_feature_names = list(best_model_feature_names)

# Get the coefficients of the best model. 
best_model_coefs = gs_results.best_estimator_._final_estimator.regressor_.coef_

# Convert the coefficients of the best model to a python list.
best_model_coefs = list(best_model_coefs)

# Build a string that represents the ridge regression equation. 
regression_equation = "Sales =" 
for feat, coef in enumerate(zip(best_model_feature_names, best_model_coefs)): 
  term = " " + str(coef[1])[:7] + "*(" + str(coef[0]) + ") + "
  regression_equation = regression_equation + term

regression_equation = regression_equation[:-2]

regression_equation


'Sales = 0.01270*(x0_Fruits and Vegetables) +  0.08816*(x1_OUT017) +  0.56196*(x1_OUT018) +  -0.6845*(x1_OUT019) +  0.98404*(x1_OUT027) +  0.11462*(x1_OUT035) +  -0.2990*(x1_OUT049) +  -0.0383*(x2_Tier 2) +  -0.7043*(x2_Tier 3) +  0.94062*(x3_Supermarket Type1) +  0.56196*(x3_Supermarket Type2) +  0.98404*(x3_Supermarket Type3) +  0.01272*(x4_Food) +  0.35044*(Outlet_Size) +  -0.0573*(2) +  0.00885*(4) '

In [ ]:

nominal_categorical_feat = ['Item_Type', 'Outlet_Identifier', 'Item_Id_Binned']
ordinal_categorical_feat = ['Item_Fat_Content', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Years_of_operation']
numeric_feat = ['Item_Weight', 'Item_Visibility', 'Item_MRP']

X = sales_df.loc[:, sales_df.columns != 'Item_Outlet_Sales']
y = sales_df.loc[:, 'Item_Outlet_Sales']


preprocess = ColumnTransformer(transformers=[("one_hot_encoder", OneHotEncoder(drop="first", sparse=False), nominal_categorical_feat),
                                             ("ordinal_encoder", OrdinalEncoder(categories=[['Not_Edible', 'Low Fat', 'Regular'], ['Small', 'Medium', 'High'],
                                                                                            ['Tier 1', 'Tier 2', 'Tier 3'], ['Grocery Store', 'Supermarket Type1', 'Supermarket Type2', 'Supermarket Type3'],
                                                                                            [4, 6, 9, 11, 14, 15, 16, 26, 28]]), ordinal_categorical_feat)],
                               remainder='passthrough')


column_list = list(range(5,35))
column_list = column_list + ['all']
  

reg_strategy = TransformedTargetRegressor()
bcox_transformer = PowerTransformer(method='box-cox')

model_pipeline = Pipeline([("preprocessing", preprocess),
                           ('feature_selector', SelectKBest(f_regression)),
                           ('regress', reg_strategy)])

parameter_grid = [{'feature_selector__k' : column_list,
                   'regress__regressor' : [LinearRegression(), Lasso(), Ridge()],
                   'regress__regressor__fit_intercept' : [False, True],
                   'regress__transformer' : [None, bcox_transformer]}]


score_types = {'r2' :'r2', 'MSE' : 'neg_mean_squared_error'}

gs = GridSearchCV(estimator=model_pipeline, param_grid=parameter_grid, scoring=score_types, refit='MSE', cv=5, n_jobs=-1)

gs.fit(X, y)

PATH = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/'
gridsearch_result_filename = 'gridsearch_models_6_Linear.pkl'
full_path = PATH + gridsearch_result_filename
with open(full_path, 'wb') as file:
  pickle.dump(gs, file)


In [ ]:
full_path = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/gridsearch_models_6_Linear.pkl'

with open(full_path, 'rb') as file:
  gs_results2 = pickle.load(file)

search_results2 = gs_results2.cv_results_
top_estimator2 = gs_results2.best_estimator_
top_score2 = gs_results2.best_score_
top_parameters2 = gs_results2.best_params_

gs_result_df2 = pd.DataFrame(search_results2)


In [52]:
results_simplified_df2 = gs_result_df2.loc[:,:].sort_values(by=['rank_test_MSE'], ignore_index=True)

results_simplified_df2.rename(columns= {'param_feature_selector__k' : 'num_features_in_model',
                                       'param_regress__regressor' : 'model_type',
                                       'param_regress__transformer' : 'target_transformation',
                                       'param_regress__regressor__fit_intercept' : 'fit_intercept'}, inplace=True)

results_simplified_df2 = results_simplified_df2.loc[:, ['num_features_in_model', 'model_type', 'target_transformation', 'fit_intercept',
                                                      'params', 'mean_test_r2', 'rank_test_r2', 'mean_test_MSE', 'rank_test_MSE']]


results_simplified_df2['target_transformation'] = results_simplified_df2['target_transformation'].fillna(value="No_Transformation")
results_simplified_df2.loc[ (results_simplified_df2['target_transformation'] != 'No_Transformation') , 'target_transformation'] = "Box-Cox"

# For easier viewing, trim model_type down so it only shows the name of the model used.
results_simplified_df2['model_type'] = results_simplified_df2['model_type'].astype(str)
results_simplified_df2['model_type'] = results_simplified_df2['model_type'].map(lambda model_string : model_string.split('(')[0])

# Negative MSE is not very interpretable. Take the absolute value and square root to get a more meaningful number. 
results_simplified_df2['RMSE'] = results_simplified_df2['mean_test_MSE'].abs().pow(1./2)

pd.set_option('display.max_rows', 500)

results_simplified_df2.head(50)

,num_features_in_model,model_type,target_transformation,fit_intercept,params,mean_test_r2,rank_test_r2,mean_test_MSE,rank_test_MSE,RMSE
0,13,LinearRegression,Box-Cox,True,"{'feature_selector__k': 13, 'regress__regresso...",0.581160,1,-1.218403e+06,1,1103.812926
1,14,LinearRegression,Box-Cox,True,"{'feature_selector__k': 14, 'regress__regresso...",0.581146,2,-1.218451e+06,2,1103.834566
2,15,LinearRegression,Box-Cox,True,"{'feature_selector__k': 15, 'regress__regresso...",0.581100,3,-1.218591e+06,3,1103.898306
3,13,Ridge,Box-Cox,True,"{'feature_selector__k': 13, 'regress__regresso...",0.580981,4,-1.218937e+06,4,1104.054906
4,12,LinearRegression,Box-Cox,True,"{'feature_selector__k': 12, 'regress__regresso...",0.580951,6,-1.218995e+06,5,1104.081241
5,14,Ridge,Box-Cox,True,"{'feature_selector__k': 14, 'regress__regresso...",0.580955,5,-1.219020e+06,6,1104.092393
6,15,Ridge,Box-Cox,True,"{'feature_selector__k': 15, 'regress__regresso...",0.580912,7,-1.219155e+06,7,1104.153533
7,11,LinearRegression,Box-Cox,True,"{'feature_selector__k': 11, 'regress__regresso...",0.580829,8,-1.219321e+06,8,1104.228762
8,12,Ridge,Box-Cox,True,"{'feature_selector__k': 12, 'regress__regresso...",0.580801,9,-1.219451e+06,9,1104.287437
9,19,LinearRegression,Box-Cox,False,"{'feature_selector__k': 19, 'regress__regresso...",0.580772,10,-1.219503e+06,10,1104.311278


In [ ]:
# Get a boolean array that indicates which features SelectKBest chose when the best model was being fit.
best_model_features_selected_alt = gs_results2.best_estimator_.named_steps['feature_selector'].get_support()

# Get the column transformer instance used when creating the best model. 
best_model_ColumnTransformer_alt =  gs_results2.best_estimator_.named_steps['preprocessing']

# Get the best models column names from the column transformer. 
best_model_column_names_alt = get_column_names_from_ColumnTransformer(best_model_ColumnTransformer_alt)

# Convert the best models column names to a np.array so we can index the list using the boolean matrix
# that indicates which features were used when fitting the best model. 
best_model_column_names_alt = np.array(best_model_column_names_alt)

# Get the names of the features used in the best model.
best_model_feature_names_alt = best_model_column_names_alt[best_model_features_selected_alt]

# Convert the names of best features from a numpy array back to a python list.
best_model_feature_names_alt = list(best_model_feature_names_alt)

# Get the coefficients of the best model. 
best_model_coefs_alt = gs_results2.best_estimator_._final_estimator.regressor_.coef_

# Convert the coefficients of the best model to a python list.
best_model_coefs_alt = list(best_model_coefs_alt)

# Build a string that represents the ridge regression equation. 
regression_equation_alt_encoding = "Sales = " 
for feat, coef in enumerate(zip(best_model_feature_names_alt, best_model_coefs_alt)): 
  term = str(coef[1])[:7] + "*(" + str(coef[0]) + ") + "
  regression_equation_alt_encoding = regression_equation_alt_encoding + term

regression_equation_alt_encoding = regression_equation_alt_encoding[:-2]

regression_equation_alt_encoding

'Sales = 0.01270*(x0_Fruits and Vegetables) + 0.08816*(x1_OUT017) + -1.6986*(x1_OUT018) + -0.0570*(x1_OUT019) + -2.4226*(x1_OUT027) + 0.11462*(x1_OUT035) + 0.01475*(x1_OUT049) + 0.01272*(x2_Food) + 0.03666*(Outlet_Size) + -0.0383*(Outlet_Location_Type) + 1.56817*(Outlet_Type) + -0.0573*(2) + 0.00885*(4) '

## Equation 1

Calculated using the following encoding: 

Nominal categorical features:  ['Item_Type', 'Outlet_Identifier', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Id_Binned']

Ordinal categorical features: ['Item_Fat_Content', 'Outlet_Size', 'Years_of_operation']

numeric features:  ['Item_Weight', 'Item_Visibility', 'Item_MRP']

In [ ]:
regression_equation

'Sales = 0.01270*(x0_Fruits and Vegetables) +  0.08816*(x1_OUT017) +  0.56196*(x1_OUT018) +  -0.6845*(x1_OUT019) +  0.98404*(x1_OUT027) +  0.11462*(x1_OUT035) +  -0.2990*(x1_OUT049) +  -0.0383*(x2_Tier 2) +  -0.7043*(x2_Tier 3) +  0.94062*(x3_Supermarket Type1) +  0.56196*(x3_Supermarket Type2) +  0.98404*(x3_Supermarket Type3) +  0.01272*(x4_Food) +  0.35044*(Outlet_Size) +  -0.0573*(2) +  0.00885*(4) '

## Equation 2

Calculated using the following encoding: 

Nominal categorical features:  ['Item_Type', 'Outlet_Identifier', 'Item_Id_Binned']

Ordinal categorical features: ['Item_Fat_Content', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Years_of_operation']

numeric features:  ['Item_Weight', 'Item_Visibility', 'Item_MRP']


Note: the only difference between this set of models and the one above is that 'Outlet type' and 'Outlet location type' moved from being nominal categorical features to ordinal categorical features. 

In [ ]:
regression_equation_alt_encoding

'Sales = 0.01270*(x0_Fruits and Vegetables) + 0.08816*(x1_OUT017) + -1.6986*(x1_OUT018) + -0.0570*(x1_OUT019) + -2.4226*(x1_OUT027) + 0.11462*(x1_OUT035) + 0.01475*(x1_OUT049) + 0.01272*(x2_Food) + 0.03666*(Outlet_Size) + -0.0383*(Outlet_Location_Type) + 1.56817*(Outlet_Type) + -0.0573*(2) + 0.00885*(4) '